In [29]:
import requests
import base64
from typing import Tuple

YOUR_EMAIL = "alvarmaciel@gmail.com"

# URL base del desafío
BASE_URL = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{YOUR_EMAIL}/register"

print(f"URL configurada: {BASE_URL}")

URL configurada: https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/alvarmaciel@gmail.com/register


In [35]:
def register_user(email: str, data: str) -> bytes:
    """
    Registra un usuario y obtiene el perfil cifrado.
    
    Args:
        email: Dirección de correo
        data: Datos adicionales (sin ; ni =)
    
    Returns:
        Bytes del IV + ciphertext
    """
    # Codificar data en base64
    data_b64 = base64.b64encode(data.encode()).decode()
    
    # Hacer POST request
    response = requests.post(
        BASE_URL,
        files={
            'email': email,
            'data': data_b64
        }
    )
    
    if response.status_code != 200:
        raise Exception(f"Error en la petición: {response.status_code}\n{response.text}")
    
    # Decodificar respuesta base64
    encrypted_profile = base64.b64decode(response.text.strip())
    
    return encrypted_profile

def split_iv_ciphertext(encrypted: bytes) -> Tuple[bytes, bytes]:
    """
    Separa el IV (primeros 16 bytes) del ciphertext.
    """
    iv = encrypted[:16]
    ciphertext = encrypted[16:]
    return iv, ciphertext

def visualize_blocks(data: bytes, block_size: int = 16, hexe=True) -> None:
    """
    Visualiza los bloques de datos.
    """
    for i in range(0, len(data), block_size):
        block = data[i:i+block_size]
        if hexe:
            print(f"Bloque {i//block_size}: {block.hex()} ({len(block)} bytes)")
        else:
            print(f"Bloque {i//block_size}: {block} ({len(block)} bytes)")

print("Funciones auxiliares definidas.")

Funciones auxiliares definidas.


In [48]:
# Probemos con un email y datos simples para entender la estructura
test_email = "a@b.com"
test_data = "TestDataAAAAAAAAAAAAAAAAAAAAAAAA"

print(f"Registrando usuario: {test_email}")
print(f"Con datos: {test_data}")
print()

# El perfil resultante será:
expected_profile = f"user={test_email};data={test_data};role=user"
print(f"Perfil esperado (plaintext): {expected_profile}")

print("\n--- Bloques a encriptar ---")
visualize_blocks(expected_profile, hexe=False)

    
print(f"Longitud: {len(expected_profile)} bytes")
print()

# Obtener el perfil cifrado
encrypted = register_user(test_email, test_data)
print(f"Perfil cifrado recibido: {len(encrypted)} bytes")
print(f"Perfil: {encrypted}")

# Separar IV y ciphertext
iv, ciphertext = split_iv_ciphertext(encrypted)
print(f"\nIV ({len(iv)} bytes): {iv.hex()}")
print(f"Ciphertext ({len(ciphertext)} bytes):")
visualize_blocks(ciphertext)

Registrando usuario: a@b.com
Con datos: TestDataAAAAAAAAAAAAAAAAAAAAAAAA

Perfil esperado (plaintext): user=a@b.com;data=TestDataAAAAAAAAAAAAAAAAAAAAAAAA;role=user

--- Bloques a encriptar ---
Bloque 0: user=a@b.com;dat (16 bytes)
Bloque 1: a=TestDataAAAAAA (16 bytes)
Bloque 2: AAAAAAAAAAAAAAAA (16 bytes)
Bloque 3: AA;role=user (12 bytes)
Longitud: 60 bytes

Perfil cifrado recibido: 80 bytes
Perfil: b'E\rH\xbaO\x94\xb55_\xc0:\x8a&\xbf\xf17\x07\x8e\rC\x81U\x0e\xb0\xae$\r\x06g\xdd\x0f\xfa9\xbe\xe8\x13@;5\xbdn:\xf5-\xdf\x00-U\x81\x87v\x9d\xfa\xe0?\xfcC+q\x9a(R\xbdV\xee\xfb\x1a\x08Fq\xe2\xc6\xc5U\x1f\xe9\x12\xc3\x97a'

IV (16 bytes): 450d48ba4f94b5355fc03a8a26bff137
Ciphertext (64 bytes):
Bloque 0: 078e0d4381550eb0ae240d0667dd0ffa (16 bytes)
Bloque 1: 39bee813403b35bd6e3af52ddf002d55 (16 bytes)
Bloque 2: 8187769dfae03ffc432b719a2852bd56 (16 bytes)
Bloque 3: eefb1a084671e2c6c5551fe912c39761 (16 bytes)


In [49]:
email = "a@b.com"
data = "TestDataAAAAAAAAAAAAAAAAAAAAAAAA"  # ajustá la cantidad
profile = f"user={email};data={data};role=user"

print(f"Profile: {profile}")
print(f"Longitud: {len(profile)}")
visualize_blocks(profile.encode(), hexe=False)

Profile: user=a@b.com;data=TestDataAAAAAAAAAAAAAAAAAAAAAAAA;role=user
Longitud: 60
Bloque 0: b'user=a@b.com;dat' (16 bytes)
Bloque 1: b'a=TestDataAAAAAA' (16 bytes)
Bloque 2: b'AAAAAAAAAAAAAAAA' (16 bytes)
Bloque 3: b'AA;role=user' (12 bytes)


In [50]:
# deltas 
original = "AA;role=user"
target = "A;role=admin"

# Calcular deltas
deltas = []
for i in range(len(original)):
    delta = ord(original[i]) ^ ord(target[i])
    deltas.append(delta)
    print(f"Posición {i}: '{original[i]}' XOR '{target[i]}' = {delta:02x}")

print(f"\nDeltas: {[f'{d:02x}' for d in deltas]}")

Posición 0: 'A' XOR 'A' = 00
Posición 1: 'A' XOR ';' = 7a
Posición 2: ';' XOR 'r' = 49
Posición 3: 'r' XOR 'o' = 1d
Posición 4: 'o' XOR 'l' = 03
Posición 5: 'l' XOR 'e' = 09
Posición 6: 'e' XOR '=' = 58
Posición 7: '=' XOR 'a' = 5c
Posición 8: 'u' XOR 'd' = 11
Posición 9: 's' XOR 'm' = 1e
Posición 10: 'e' XOR 'i' = 0c
Posición 11: 'r' XOR 'n' = 1c

Deltas: ['00', '7a', '49', '1d', '03', '09', '58', '5c', '11', '1e', '0c', '1c']


In [51]:
original = b'AA;role=user'
target = b"A;role=admin"
mask = bytes(a ^ b for a, b in zip(original, target))
mask

b'\x00zI\x1d\x03\tX\\\x11\x1e\x0c\x1c'

In [52]:
iv

b'E\rH\xbaO\x94\xb55_\xc0:\x8a&\xbf\xf17'

In [53]:
ciphertext

b'\x07\x8e\rC\x81U\x0e\xb0\xae$\r\x06g\xdd\x0f\xfa9\xbe\xe8\x13@;5\xbdn:\xf5-\xdf\x00-U\x81\x87v\x9d\xfa\xe0?\xfcC+q\x9a(R\xbdV\xee\xfb\x1a\x08Fq\xe2\xc6\xc5U\x1f\xe9\x12\xc3\x97a'

In [54]:
# Extraer bloques del ciphertext
block0 = ciphertext[0:16]
block1 = ciphertext[16:32]
block2 = ciphertext[32:48]
block3 = ciphertext[48:]

# Aplicar máscara al bloque 1
block2_modified = bytes(b ^ m for b, m in zip(block2[:12], mask)) + block2[12:]

# Reconstruir ciphertext modificado
modified_ciphertext = block0 + block1 + block2_modified + block3

# Reconstruir mensaje completo
modified_encrypted = iv + modified_ciphertext

print(f"Bloque 1 original: {block1.hex()}")
print(f"Bloque 1 modificado: {block1_modified.hex()}")

Bloque 1 original: 39bee813403b35bd6e3af52ddf002d55
Bloque 1 modificado: 6e3d0cc5b83320729b40dc914fbf9252


In [55]:
# Reconstruir mensaje completo (IV + ciphertext)
modified_encrypted = iv + modified_ciphertext

# Codificar en base64
modified_b64 = base64.b64encode(modified_encrypted).decode()

print(f"Mensaje modificado (base64):")
print(modified_b64)
print(f"\nLongitud total: {len(modified_encrypted)} bytes")

Mensaje modificado (base64):
RQ1Iuk+UtTVfwDqKJr/xNweODUOBVQ6wriQNBmfdD/o5vugTQDs1vW469S3fAC1Vgf0/gPnpZ6BSNX2GKFK9Vu77GghGceLGxVUf6RLDl2E=

Longitud total: 80 bytes


In [56]:
challenge_response = requests.post(
    f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{YOUR_EMAIL}/answer",
    files = { "message":modified_b64}
)
print(f"Status code: {challenge_response.status_code}")
print(challenge_response.text)

Status code: 200
¡Ganaste!

